# TensorFlow의 기본 사용법
(참고: https://codeonweb.com/entry/5f15bf8e-d704-49e0-909a-db4450433b74)

TensorFlow를 사용하기 위해서는 TensorFlow가 어떻게 다음과 같은 일을 수행하는지 이해해야 합니다:

* 어떻게 계산을 그래프화하는가.
* 어떻게 세션(Session)에서 그래프를 실행하는가.
* 어떻게 데이터를 텐서로 표현하는가.
* 어떻게 변수(Variable)로 상태를 관리하는가.
* 어떻게 임의의 연산(operations)으로부터 데이터를 얻거나 저장하기 위해 피드(feed)와 페치(fetch)를 사용하는가.

---
### 개요
 
TensorFlow는 계산(computation)을 그래프(graph)로 표현하는 프로그래밍 시스템입니다. 
그래프의 노드(node)는 연산(ops, operations의 줄임)이라고 부릅니다. 
하나의 연산은 0개 이상의 Tensor를 취해 어떤 계산을 한 뒤 0개 이상의 Tensor를 다시 반환합니다. 
Tensor는 형태를 가진 다차원 배열이라고 할 수 있습니다.  
예를 들어, 여러 개의 이미지를 실수(floating point number)로 이루어진 4-D 배열([이미지, 높이, 너비, 채널]) 형태로 표현해볼 수 있을 것입니다.  

---

TensorFlow에서 그래프는 계산을 기술한 것입니다. 
어떤 것이든 계산하기 위해서는 반드시 그래프를 Session에 올려야 합니다.  
Session은 CPU나 GPU 같은 Devices에 그래프 연산을 올린 뒤 연산을 실행할 수 있는 메소드를 제공합니다.  
이 메소드는 연산에 의해 생성된 텐서를 반환하는데, 이 텐서의 형태는 파이썬에서는 numpy의 ndarray 객체, C와 C++에서는 tensor flow::Tensor 개체입니다.

---
### 계산 그래프(computation graph)

TensorFlow 프로그램은 크게 두 가지 단계로 구성되어 있습니다.  

구성(construction) 단계에서는 그래프를 조립(assemble)하고, 실행(execution) 단계에서는 세션을 이용해 그래프의 연산을 실행합니다.  

예를 들어, 구성 단계에서 신경망을 나타내고 훈련할 수 있는 그래프를 생성하고, 실행 단계에서 반복적으로 그래프의 훈련 연산을 실행하는 식의 진행은 흔합니다.  
TensorFlow는 C, C++, 파이썬 프로그램에서 사용할 수 있습니다. 아직까지는 파이썬 라이브러리를 사용하는 편이 그래프를 조립하기에 훨씬 용이합니다.  
C나 C++ 라이브러리가 제공하지 않는 방대한 도움 함수를 사용할 수 있기 때문입니다.  
세션 라이브러리는 세 언어에서 같은 수준의 기능을 제공합니다.

---
##### 그래프 만들기

그래프를 만들기 위해서는 Constant와 같이 어떤 입력도 필요로 하지 않는 연산(소스 연산)으로 시작하십시오.  
그리고 그 출력을 계산을 수행하는 다른 연산에 넘겨줍니다.

파이썬 라이브러리의 연산 생성자(ops constructor)는 생성된 연산의 출력을 나타내는 객체를 반환합니다.  
우리는 이 객체를 다른 연산 생성자의 입력으로 넘겨줄 수 있습니다.

TensorFlow 파이썬 라이브러리는 기본 그래프를 가지고 있는데, 연산 생성자가 이 그래프에 노드를 추가합니다.  
대부분의 경우에는 기본 그래프를 쓰는 것으로 충분합니다. 여러 개의 그래프를 관리하는 법이 궁금하다면 Graph 클래스 문서를 참고하십시오.

(class tf.Graph: https://www.tensorflow.org/versions/r0.7/api_docs/python/framework.html#Graph)

In [1]:
import tensorflow as tf

In [2]:
# 1x2 행렬을 만드는 tf.constant
# 이 연산자는 기본 그래프의 노드로 추가됩니다.
matrix1 =  tf.constant([[3., 3.]])
print(matrix1)

Tensor("Const:0", shape=(1, 2), dtype=float32)


In [3]:
# 2x1 행렬을 만드는 또 다른 Constant를 생성합니다.
matrix2 = tf.constant([[2.],[2.]])
print(matrix2)

Tensor("Const_1:0", shape=(2, 1), dtype=float32)


In [4]:
# 'matrix1'과 'matrix2'를 입력으로 받는 Matmul 연산을 생성합니다.
# 반환값인 'product'는 행렬을 곱한 결과를 나타냅니다.
product = tf.matmul(matrix1, matrix2)
print(product)

Tensor("MatMul:0", shape=(1, 1), dtype=float32)


기본 그래프는 이제 두 개의 constant() 연산과 하나의 matmul() 연산을 합해, 총 세 개의 노드를 가지게 되었습니다.  
실제로 행렬을 곱해서 결과를 얻으려면 세션에 그래프를 올려야 합니다.

---

##### 그래프를 세션에 올리기


그래프를 세션에 올리려면 먼저 Session 객체를 만들어야 합니다.  
아무런 인자를 넘기지 않으면 세션 생성자는 기본 그래프를 사용하게 됩니다.  
세션 API가 궁금하다면 Session 클래스 문서를 참고하세요.

(class tf.Session: https://www.tensorflow.org/versions/r0.7/api_docs/python/client.html#session-management)

In [5]:
# 기본 그래프로 세션을 생성.
sess = tf.Session()

# matmul 연산을 실행하려면 matmul 연산의 출력을 나타내는 'product'를
# 인자로 넘겨주면서 세션의 'run()' 메소드를 호출합니다. 이렇게 호출하면
# matmul 연산의 출력을 다시 얻고 싶다는 뜻입니다.
#
# 연산이 필요로 하는 모든 입력은 세션에 의해 자동으로 실행됩니다.
# 일반적으로 병렬적으로 실행되지요.
#
# 따라서 'run(product)' 호출은 그래프 내의 세 개 연산을 실행하게 됩니다.
# 두 개의 상수와 matmul이 바로 그 연산이지요.
#
# 연산의 출력은 'result'에 numpy의 `ndarray` 객체 형태로 저장됩니다. 
result = sess.run(product)
print(result)
# ==> [[ 12.]]

# 다 끝났으면 세션을 닫아줍니다.
sess.close()

[[ 12.]]


자원을 시스템에 돌려주려면 세션을 닫아야 합니다. 

"with" 블록을 이용해서 Session에 들어갈 수도 있습니다. 이 경우 with 블록이 끝나면 Session이 자동으로 닫히게 됩니다.

In [6]:
with tf.Session() as sess:
  result = sess.run([product])
  print(result)

[array([[ 12.]], dtype=float32)]


TensorFlow로 구현하면 그래프 정의를 실행 가능한 연산으로 바꾸어 가용한 컴퓨터 자원(CPU, GPU 등)에 분배합니다.  
일반적으로 CPU나 GPU를 구체적으로 지정할 필요는 없습니다.   
여러분의 시스템에 GPU가 하나 있다면, TensorFlow는 GPU에 가능한 많은 연산을 넘겨줍니다.

만약 두 개 이상의 GPU가 있다면 명시적으로 정해 줄 필요가 있습니다.  
with...Device 구문을 사용해서 연산에 어떤 CPU 혹은 GPU를 사용할 것인지 지정할 수 있습니다:

In [7]:
with tf.Session() as sess:
  with tf.device("/gpu:1"):
    matrix1 = tf.constant([[3., 3.]])
    matrix2 = tf.constant([[2.],[2.]])
    product = tf.matmul(matrix1, matrix2)

장치는 문자열로 나타냅니다. 현재 지원되는 장치는:

* "/cpu:0": 여러분 컴퓨터의 CPU.
* "/gpu:0": 여러분 컴퓨터의 첫 번째 GPU.
* "/gpu:1": 여러분 컴퓨터의 두 번째 GPU, 등등.

GPU와 TensorFlow에 대한 더 자세한 사항은 GPU 사용하기 문서를 참고하십시오.

(Using GPU: https://www.tensorflow.org/versions/r0.7/how_tos/using_gpu/index.html)

---

### 대화형으로 사용(Interactive Usage)

이 문서에서 본 파이썬 예제에서는 그래프를 Session에 올린 뒤 Session.run() 메소드를 이용해 연산을 실행했습니다.  
IPython과 같은 대화형 파이썬 환경에서 보다 쉽게 사용하려면,InteractiveSession 클래스를 통해 Tensor.eval()과 Operation.run() 메소드를 호출할 수도 있습니다.  
그러면 세션을 위해 변수를 미리 생성할 필요가 없습니다.



In [8]:
# 대화형 TensorFlow 세션에 들어갑니다.
import tensorflow as tf
sess = tf.InteractiveSession()

x = tf.Variable([1.0, 2.0])
a = tf.constant([3.0, 3.0])

# 'x'의 초기화 연산의 run() 메소드를 사용해서 'x'를 초기화합니다.
x.initializer.run()

# 'x'에서 'a'를 빼기 위해 연산을 추가. 실행하고 결과를 출력합니다.
sub = tf.sub(x, a)
print(sub.eval())
# ==> [-2. -1.]

# 다 끝났으면 세션을 닫아줍니다.
sess.close()

[-2. -1.]


---

### 텐서

TensorFlow 프로그램은 텐서(tensor) 데이터 구조를 사용해서 모든 데이터를 나타냅니다.  
계산 그래프에서 연산을 할 때에도 오직 텐서만 주고 받습니다.  
TensorFlow의 텐서는 n 차원의 배열이나 리스트라고 생각하면 됩니다.  
텐서는 정적 타입(static type), 랭크(rank), 모양(shape)의 속성을 가집니다.  
TensorFlow가 어떻게 이 개념들을 다루는지 알고 싶다면 Rank, Shape, and Type 문서를 참고하십시오.

(Tensor Ranks, Shapes,and Types: https://www.tensorflow.org/versions/r0.7/resources/dims_types.html)

---

### 변수

변수는 그래프가 실행되는 동안 어떤 상태를 담고 있습니다.  
다음 예제는 간단한 계수 역할을 하는 변수를 보여주고 있습니다.  
더 자세한 사항은 Variables를 참고하세요.

(Variables: Creation, Initialization, Saving, and Loading: https://www.tensorflow.org/versions/r0.7/how_tos/variables/index.html)

In [9]:
import tensorflow as tf
# 변수를 하나 생성하고 스칼라 값인 0으로 초기화합니다.
state = tf.Variable(0, name="counter")

# one을 state에 더하는 연산을 생성합니다.
one = tf.constant(1)
new_value = tf.add(state, one)
update = tf.assign(state, new_value)

# 변수는 그래프가 올라간 뒤 'init' 연산을 실행해서 반드시 초기화 되어야 합니다.
# 그 전에 먼저 'init' 연산을 그래프에 추가해야 합니다.
init_op = tf.initialize_all_variables()

# 그래프를 올리고 연산을 실행합니다.
with tf.Session() as sess:
  # 'init' 연산을 실행합니다.
  sess.run(init_op)
  # 'state'의 초기값을 출력합니다.
  print(sess.run(state))
  # 'state'를 갱신하는 연산을 실행하고 'state'를 출력합니다.
  for _ in range(3):
    sess.run(update)
    print(sess.run(state))


0
1
2
3


이 코드에 등장한 assign() 연산은 add() 연산과 마찬가지로 표현 그래프(expression graph)의 일종입니다.   
그래서 run()으로 그 표현을 실행하기 전에는 실제 대입 연산을 수행하지 않습니다.

일반적으로 통계 모델의 파라미터를 변수로 나타냅니다.   
예를 들어, 신경망의 가중치(weight)를 텐서 변수에 저장할 수 있습니다.   
훈련 과정이 진행되면 훈련 그래프를 반복적으로 실행해서 이 텐서를 갱신하게 됩니다.  

---

### 가져오기(Fetchs)

연산의 출력을 가져오기 위해서는 Session 객체에서 가져오고 싶은 텐서를 run()에 인자로 넘겨서 그래프를 실행해야 합니다.  
이전 예제에서 우리는 state라는 한 개의 노드를 가져왔지요.  
하지만 여러 개의 텐서를 가져올 수도 있습니다:

In [10]:
import tensorflow as tf

input1 = tf.constant(3.0)
input2 = tf.constant(2.0)
input3 = tf.constant(5.0)
intermed = tf.add(input2, input3)
mul = tf.mul(input1, intermed)

with tf.Session() as sess:
  result = sess.run([mul, intermed])
  print(result)

[21.0, 7.0]


요청된 텐서들의 출력값을 만드는 데 관여하는 모든 연산은 (요청된 텐서 하나 당 한 번이 아니라) 한 번만 실행됩니다.

---

### 피드(Feeads)

지금까지 본 예제에서는 텐서의 값을 Conatants와 Variables에 미리 저장한 뒤 계산 그래프에 넘겨줍니다.  
TensorFlow는 그래프 내에서 직접 텐서의 값을 할당할 수 있는 방법도 제공하고 있습니다.


피드(feed)는 연산의 출력을 지정한 텐서 값으로 임시 대체합니다.  
임시로 사용할 피드 데이터는 run()의 인자로 넘겨줄 수 있습니다.  
피드 데이터는 run을 호출할 때 명시적 인자로 넘겨질 때만 사용됩니다.  
가장 흔한 사용법은 tf.placeholder()를 사용해 특정 연산을 "피드" 연산으로 지정하는 것입니다.

In [11]:
import tensorflow as tf

input1 = tf.placeholder(tf.float32)
input2 = tf.placeholder(tf.float32)
output = tf.mul(input1, input2)

with tf.Session() as sess:
  print(sess.run([output], feed_dict={input1:[7.], input2:[2.]}))

[array([ 14.], dtype=float32)]


placeholder() 연산은 피드 데이터를 제공하지 않으면 오류를 일으킵니다. 

---

더 큰 규모의 피드 예제를 보고 싶다면 MNIST fully-connected feed tutorial을 참고하십시오.  

(MNIST fully-connected feed tutorial: https://www.tensorflow.org/versions/r0.7/tutorials/mnist/tf/index.html)
